In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=12, threads_per_worker=1)
from dask import bag as db
import gcsfs

project_id = 'ultra-compound-304316'
fs = gcsfs.GCSFileSystem(project=project_id)

In [2]:
X_small_train = db.read_text('gcs://dsp-pj1/project1/files/X_small_train.txt').map(lambda x:x.strip().split()).flatten()
X_small_test = db.read_text('gcs://dsp-pj1/project1/files/X_small_test.txt').map(lambda x:x.strip().split()).flatten()
y_small_train = db.read_text('gcs://dsp-pj1/project1/files/y_small_train.txt').map(lambda x:x.strip().split()).flatten()
y_small_test = db.read_text('gcs://dsp-pj1/project1/files/y_small_test.txt').map(lambda x:x.strip().split()).flatten()

In [3]:
from dask import dataframe as df
import pandas as pd
hash_key = X_small_train.compute()

data = df.read_csv('gcs://dsp-pj1/features/small_train/'+hash_key[0]+'.csv/part-*.csv', header = None, names = ['byte', 'freq'])
datar = data.compute()
datar = datar.transpose()
datar.reset_index(inplace=True)
datar = datar.drop(columns=['index'],axis=1)
datar.colucmns = datar.iloc[0]
datar = datar.drop([0])
datar = datar.drop(columns=['??'])
datap = datar

for hashes in hash_key[1:]:
    data1 = df.read_csv('gcs://dsp-pj1/features/small_train/'+hashes+'.csv/part-*.csv', header = None, names = ['byte', 'freq'])
    datar1 = data1.compute()
    datar1 = datar1.transpose()
    datar1.reset_index(inplace=True)
    datar1 = datar1.drop(columns=['index'],axis=1)
    datar1.columns = datar1.iloc[0]
    datar1 = datar1.drop([0])
    try:
        datar1 = datar1.drop(columns=['??'])
    except:
        pass
    datap = datap.append(datar1,ignore_index=True)

datap['targets'] = y_small_train.compute()


In [4]:
datap = datap.dropna()
datap

,00,01,02,03,04,05,06,07,08,09,...,F7,F8,F9,FA,FB,FC,FD,FE,FF,targets
0,7217,2161,2021,1972,1928,1895,1933,1987,2015,1981,...,2055,2164,2021,2049,1946,2052,1958,1982,2565,6
1,11363,5531,3232,3340,3387,3178,3328,3265,3285,3307,...,3104,3045,3279,3212,3158,3066,3265,3142,6934,3
2,30316,882,538,710,733,409,373,449,500,494,...,282,457,277,302,285,373,288,377,3553,1
4,48678,3191,1448,1737,2247,686,681,619,1578,546,...,586,930,454,448,417,703,431,579,5079,1
5,100356,6211,3200,4236,4766,6071,4232,1369,6102,1079,...,1307,2247,1015,951,1398,2981,1240,2640,26982,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,10684,5439,3241,3312,3274,3174,3163,3273,3283,3236,...,3165,3173,3188,3185,3248,3148,3057,3215,6715,3
375,8395,4207,3110,3256,3065,3086,3100,3141,3196,3106,...,3072,3022,3182,3111,3094,3075,3067,3051,5115,3
376,46879,15764,550223,10967,12338,10278,11473,10768,10714,297,...,558,11489,334,209,150,11754,212,8953,58271,2
377,25767,689,227,357,245,295,161,210,194,143,...,482,928,697,696,755,892,555,593,3458,8


In [5]:
def return_summary(frame):
    means = [frame[k].mean() for k in frame.columns if k!='targets']
    stds = [frame[k].std() for k in frame.columns if k!='targets']
    lens = frame.shape[0]
    return {'means':means,'stds':stds,'lens':lens}

In [6]:
from math import sqrt
from math import pi
from math import exp


In [7]:
def return_class_summary(dataset_frame):
    classes = sorted(dataset_frame['targets'].unique().astype('int32'))
    class_summaries = []
    for i in classes:
        class_summaries.append(return_summary(datap[datap['targets']==str(i)]))
        class_summaries[-1]['class'] = str(i)
    return class_summaries
sums = return_class_summary(datap)

In [8]:
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

In [9]:
from math import log10, log2
def class_probabilities(summaries, row):
    total_rows = sum([summaries[i]['lens'] for i in range(0,len(summaries))])
    probabilities = dict()
    for srs in summaries:
        probabilities[srs['class']] = -log10(srs['lens'] / total_rows )
        for jj in range(0,len(srs['means'])):
            try:
                probabilities[srs['class']] -= log10(calculate_probability(row[jj],srs['means'][jj],srs['stds'][jj]))
            except:
                probabilities[srs['class']] -= 0
    return probabilities

class_probabilities(sums,datap.loc[0].astype('int32').tolist())

<ipython-input-8-6bca935ce75f>:2: RuntimeWarning: divide by zero encountered in double_scalars
  exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
<ipython-input-8-6bca935ce75f>:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (1 / (sqrt(2 * pi) * stdev)) * exponent
<ipython-input-8-6bca935ce75f>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (1 / (sqrt(2 * pi) * stdev)) * exponent


{'1': 980.8763926221742,
 '2': 1186.0093324156662,
 '3': 1060.0858335983958,
 '4': 1459.266652275943,
 '5': nan,
 '6': 864.8995728174572,
 '7': 1698.67665137056,
 '8': 2969.882767963257,
 '9': 956.7441279630515}

In [10]:
import operator
def ret_class(clss_probs):
    return min(clss_probs.items(), key=operator.itemgetter(1))[0]

clp = class_probabilities(sums,datap.iloc[0].astype('int32').tolist())
ret_class(clp)

<ipython-input-8-6bca935ce75f>:2: RuntimeWarning: divide by zero encountered in double_scalars
  exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
<ipython-input-8-6bca935ce75f>:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (1 / (sqrt(2 * pi) * stdev)) * exponent
<ipython-input-8-6bca935ce75f>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (1 / (sqrt(2 * pi) * stdev)) * exponent


'6'

In [11]:
def NB_pred(class_summary,X_test):
    y_pred = []
    for sample_no in range(0,X_test.shape[0]):
        clp = class_probabilities(sums,X_test.iloc[sample_no].astype('int32').tolist())
        y_pred.append(ret_class(clp))
        
    return y_pred

In [12]:
hash_key2 = X_small_test.compute()

data = df.read_csv('gcs://dsp-pj1/features/small_test/'+hash_key2[0]+'.csv/part-*.csv', header = None, names = ['byte', 'freq'])
datar = data.compute()
datar = datar.transpose()
datar.reset_index(inplace=True)
datar = datar.drop(columns=['index'],axis=1)
datar.columns = datar.iloc[0]
datar = datar.drop([0])
datar = datar.drop(columns=['??'])
datap_test = datar

for hashes in hash_key2[1:]:
    data1 = df.read_csv('gcs://dsp-pj1/features/small_test/'+hashes+'.csv/part-*.csv', header = None, names = ['byte', 'freq'])
    datar1 = data1.compute()
    datar1 = datar1.transpose()
    datar1.reset_index(inplace=True)
    datar1 = datar1.drop(columns=['index'],axis=1)
    datar1.columns = datar1.iloc[0]
    datar1 = datar1.drop([0])
    try:
        datar1 = datar1.drop(columns=['??'])
    except:
        pass
    datap_test = datap_test.append(datar1,ignore_index=True)


datap_test['targets'] = y_small_test.compute()

In [13]:
datap_test = datap_test.dropna()
datap_test

,00,01,02,03,04,05,06,07,08,09,...,F7,F8,F9,FA,FB,FC,FD,FE,FF,targets
0,64872,2401,1909,2122,2268,1729,1723,1846,2273,1822,...,1582,1788,1541,1526,1562,1767,1548,1796,4570,6
1,10530,5499,3200,3269,3281,3165,3320,3245,3295,3163,...,3126,3227,3240,3065,3213,3205,3207,3254,6678,3
2,4769,1134,1165,1346,1233,1162,1113,1151,1135,1085,...,1199,1148,1098,1089,1158,1139,1155,1168,1133,4
3,9896,5406,3174,3306,3385,3355,3182,3285,3309,3343,...,3174,3133,3238,3112,3143,3232,3244,3207,6778,3
4,11494,5654,3300,3299,3276,3266,3326,3266,3159,3328,...,3256,3173,3330,3161,3187,3234,3191,3158,6828,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,92866,6254,2872,2529,4768,1611,1685,1182,5867,1040,...,952,2142,697,774,582,1859,846,1713,24666,1
165,14296,8523,6252,6212,6377,6185,6072,6220,6012,6363,...,6142,6159,6031,6246,6098,6079,5922,6653,9889,3
166,3579,1460,1409,1366,1342,1385,1621,1428,1435,1397,...,1419,1580,1358,1371,1395,1346,1329,1397,1805,6
167,7343,900,880,571,557,744,544,447,559,454,...,503,538,507,571,883,903,486,476,866,8


In [14]:
def acc(y_pred, y_test):
    acc = 0
    for i in range(0,len(y_test)):
        if y_pred[i] == y_test[i]:
            acc+=1
    return acc/len(y_test)*100

In [15]:
acc(NB_pred(sums, datap_test),datap_test['targets'].tolist())

<ipython-input-8-6bca935ce75f>:2: RuntimeWarning: divide by zero encountered in double_scalars
  exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
<ipython-input-8-6bca935ce75f>:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (1 / (sqrt(2 * pi) * stdev)) * exponent
<ipython-input-8-6bca935ce75f>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (1 / (sqrt(2 * pi) * stdev)) * exponent


56.886227544910184